In [ ]:
# Step 1: Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import neurolab as nl
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.optimizers import RMSprop
from rbflayer import RBFLayer, InitCentersRandom
from kmeans_initializer import InitCentersKMeans
import pyswarms as ps
from pyswarms.single.global_best import GlobalBestPSO
import csv, joblib, shap

# Step 2: Import and Preprocess dataset
df = pd.read_csv("Trapping Index Dataset.csv")
X = df.iloc[:, :8].values #Exttract the 8 input features
y = df.iloc[:, 8].values #Extract ouput feature (NB:Use for RTI models)
y = df.iloc[:, 9].values #Extract ouput feature (NB:Use for STI models)
y = y.reshape(-1,1)
# Split data into training and test sets (80% train set and 20% test set)
X_train_actual, X_test_actual, y_train_actual, y_test_actual = train_test_split(X, y, test_size=0.2, random_state=42)
# Scaling dataset to prevent numerical overflow
scaler = MinMaxScaler(feature_range=(-1,1))
X_train = scaler.fit_transform(X_train_actual)
X_test = scaler.fit_transform(X_test_actual)
y_train = scaler.fit_transform(y_train_actual)
y_test = scaler.fit_transform(y_test_actual)
y_train=y_train.reshape(-1)

# Step 3: Build Predictive Models
# Random search optimized Hyrbid Models
## 1. SVR-RS Hybrid Model
### Define the model
svr_model = SVR(kernel = 'rbf')
### parameter search range
c_range = np.logspace(1, 3, 100)
gamma_range = np.logspace(-2, 1, 20)
epsilon_range = np.logspace(-3, 1, 20)
param_grid = dict(C = c_range, gamma=gamma_range,epsilon = epsilon_range )
### Deploy randomsearchcv
random_search = RandomizedSearchCV(svr_model, param_distributions = param_grid, n_iter = 100, cv=5,
                                      scoring ='neg_root_mean_squared_error',
                                      refit= False, random_state=42)
random_search.fit(X_train,y_train)
### Obtain Optimal Hyperparameters
best_params = random_search.best_params_
best_C= best_params['C']
best_gamma= best_params['gamma']
best_epsilon= best_params['epsilon']
print('C:', best_C, 'gamma:', best_gamma, 'epsilon:', best_epsilon)
### Train with the best paramaters
RS_svr_model = SVR(kernel = 'rbf', C=best_C, gamma=best_gamma, epsilon = best_epsilon)
RS_svr_model.fit(X_train, y_train)

## 2. RF-RS Hybrid Model
### Define model
rf_model = RandomForestRegressor()
### Create the param grid for RF
param_grid = {'n_estimators' : np.arange(100,1000,10),
              'max_depth': np.arange(2,30),
              'min_samples_split' : np.arange(2,25),
              'min_samples_leaf' : np.arange(1,25)
}
### Deploy randomsearchcv
random_search = RandomizedSearchCV(rf_model, param_distributions = param_grid, n_iter = 100, cv=5,
                                      scoring ='neg_root_mean_squared_error',
                                      refit=False, n_jobs=-1, random_state=101)
random_search.fit(X_train,y_train)
### Obtain Optimal Hyperparameters
best_params = random_search.best_params_
best_n_estimators= int(best_params['n_estimators'])
best_max_depth = int(best_params['max_depth'])
best_min_samples_split = int(best_params['min_samples_split'])
best_min_samples_leaf = int(best_params['min_samples_leaf'])
print('n_estimators:', best_n_estimators, 'max_depth:', best_max_depth, 'min_samples_split:',
      best_min_samples_split,'min_samples_leaf:',best_min_samples_leaf)
### Train with the best paramaters
RS_rf_model = RandomForestRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth,
                                    min_samples_split=best_min_samples_split,
                                    min_samples_leaf=best_min_samples_leaf)
RS_rf_model.fit(X_train, y_train)

## 3. XGB-RS Hybrid Model
## Define model
xgb_model = XGBRegressor()
# Create the param grid for XGB
param_grid = {'n_estimators' : np.arange(100,1000,10),
              'max_depth': np.arange(3,30),
              'gamma': np.logspace(-2, 1, 20),
              'learning_rate': np.logspace(-2, 1, 20)
}
## Deploy randomsearchcv
random_search = RandomizedSearchCV(xgb_model, param_distributions = param_grid, n_iter = 100, cv=5,
                                      scoring ='neg_root_mean_squared_error',
                                      refit=False, random_state=2024)
random_search.fit(X_train,y_train)
### Obtain Optimal Hyperparameters
best_params = random_search.best_params_
best_n_estimators= best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_gamma = best_params['gamma']
best_learning_rate = best_params['learning_rate']
print('n_estimators:', best_n_estimators, 'max_depth:', best_max_depth, 'gamma:',
      best_gamma,'learning_rate:',best_learning_rate)
## Train with the best paramaters
RS_xgb_model = XGBRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth,
                                    gamma=best_gamma,
                                    learning_rate=best_learning_rate)
RS_xgb_model.fit(X_train, y_train)

#Particle Swarm Optimized Hybrid Models
## 1. SVR-PSO
### Define objective function for pso
def svr_objective_function(params):
    C = params[0,0]
    gamma = params[0,1]
    epsilon = params[0,2]
    svr_model = SVR(kernel='rbf', C=C, gamma=gamma,epsilon=epsilon)
    scores = cross_val_score(svr_model, X_train, y_train, cv=5,scoring='neg_mean_squared_error')
    scores = -scores
    mse = np.mean(scores)
    return mse
### Initialize PSO parameters
lb= [100,0.01,0.001]
ub =[1000,1,1]
bounds = (lb,ub)
options = {'c1': 2.05, 'c2': 2.05, 'w': 0.9}
max_iterations = 200
### Create optimizer
optimizer = GlobalBestPSO(n_particles=50, dimensions=3, options=options, bounds=bounds)
### Finding Optimal Hyperameters with PSO
best_cost, best_params= optimizer.optimize(svr_objective_function, iters=max_iterations,)
best_C = best_params[0]
best_gamma = best_params[1]
best_epsilon = best_params[2]
print("Optimal SVR parameters: C=", best_C, ", epsilon=",best_epsilon, ", gamma=", best_gamma)
### Train model with optimal hyperparameters
PSO_svr_model = SVR(C=best_C, epsilon= best_epsilon, kernel = 'rbf', gamma=best_gamma)
PSO_svr_model.fit(X_train, y_train)

### 2. RF-PSO
### Define objective function for pso
def rf_objective_function(params):
  n_estimators = params[0,0]
  max_depth = params[0,1]
  min_samples_split = params[0,2]
  min_samples_leaf = params[0,3]
  rf_model = RandomForestRegressor(n_estimators = int(n_estimators),
                                   max_depth = int(max_depth),min_samples_split = int(min_samples_split),
                                   min_samples_leaf = int(min_samples_leaf), random_state = 42)
  scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
  scores = -scores
  mse = np.mean(scores)
  return mse
### Initialize PSO parameters
lb = [100,2,2,1]
ub = [1000,30,25,25]
bounds = (lb,ub)
options = {'c1':1.5, 'c2':2, 'w':0.8}
max_iterations = 200
### Create optimizer
optimizer=ps.single.GlobalBestPSO(n_particles=100, dimensions=4, options=options, bounds=bounds)
### Find Optimal Hyperameters with PSO
best_cost, best_params = optimizer.optimize(rf_objective_function,iters=max_iterations)
best_n_estimators= best_params[0]
best_max_depth= best_params[1]
best_min_samples_split= best_params[2]
best_min_samples_leaf= best_params[3]
print("Optimal RF parameters: n_estimators=", best_n_estimators, "max_depth=" ,best_max_depth,
      "min_samples_split =" ,best_min_samples_split, "min_samples_leaf =", best_min_samples_leaf)
### Train model with optimal hyperparameters
PSO_rf_model = RandomForestRegressor(n_estimators = int(best_n_estimators),max_depth = int(best_max_depth),
                                     min_samples_split = int(best_min_samples_split),
                                     min_samples_leaf = int(best_min_samples_leaf),random_state = 42)
PSO_rf_model.fit(X_train, y_train)

## 3. XGB-PSO
### Define objective function for pso
def xgb_objective_function(params):
    n_estimators = params[0,0]
    max_depth = params[0,1]
    gamma = params[0,2]
    learning_rate = params[0,3]
    xgb_model = XGBRegressor(n_estimators = int(n_estimators),max_depth = int(max_depth),
                                     gamma = gamma,
                                     learning_rate = learning_rate, random_state = 42)
    scores = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
    scores = -scores
    mse = np.mean(scores)
    return mse
### Initialize PSO parameters
lb = [100,3,0.01,0.1]
ub = [1000,30,1,1]
bounds = (lb,ub)
options = {'c1':2.05, 'c2':2.05, 'w':0.9}
max_iterations = 200
### Create optimizer
optimizer=ps.single.GlobalBestPSO(n_particles=150, dimensions=4, options=options, bounds=bounds)
# Find Optimal Hyperameters with PSO
best_cost, best_params = optimizer.optimize(xgb_objective_function,iters=max_iterations)
best_n_estimators= best_params[0]
best_max_depth= best_params[1]
best_min_gamma= best_params[2]
best_learning_rate= best_params[3]
print("Optimal RF parameters: n_estimators=", best_n_estimators, "max_depth=" ,best_max_depth,
      "min_gamma =" ,best_min_gamma, "learning_rate =", best_learning_rate)
#Train model with optimal hyperparameters

PSO_xgb_model = XGBRegressor(n_estimators = int(best_n_estimators),
                                     max_depth = int(best_max_depth),
                                     min_gamma = best_min_gamma,
                                     learning_rate = best_learning_rate,random_state = 42)
PSO_xgb_model.fit(X_train, y_train)

# Default RBFNN model
## creating RBFLayer with centers found by KMeans clustering
n_rbf_units = 10  # Adjust the number of RBF units based on your problem
init_centers_kmeans = InitCentersKMeans(X_train)  # Instantiate InitCentersKMeans with your training data
rbf_layer = RBFLayer(n_rbf_units,initializer=init_centers_kmeans, betas=2.0, input_shape=(8,))
## Define the model architecture
dft_rbfnn_model = Sequential()
dft_rbfnn_model.add(rbf_layer)
dft_rbfnn_model.add(Dense(1))  # Output layer
dft_rbfnn_model.compile(loss='mean_squared_error', optimizer=RMSprop())
## Train the model
dft_rbfnn_model.fit(X_train, y_train, epochs=100, batch_size=50,verbose=1)

# Step 4: Evaluate the model
## 1. On trainset
train_prediction = model_name.predict(X_train) #NB: replace model_name with actual model name( eg: RS_xgb_model or PSO_xgb_model)
train_prediction = train_prediction.reshape(-1, 1)
train_prediction_actual = scaler.inverse_transform(train_prediction)
r2_train = r2_score(y_train_actual, train_prediction_actual)
mse_train = mean_squared_error(y_train_actual,train_prediction_actual)
rmse_train = np.sqrt(mse_train)
mae_train = mean_squared_error(y_train_actual, train_prediction_actual)
percentage_deviations = ((y_train_actual-train_prediction_actual)/y_train_actual)*100
mpe_train = np.mean(percentage_deviations)
mape_train = np.mean(np.abs(percentage_deviations))

## 2. On testset
test_prediction = model_name.predict(X_test) #NB: replace model_name with actual model name( eg: RS_xgb_model or PSO_xgb_model)
test_prediction = test_prediction.reshape(-1, 1)
test_prediction_actual = scaler.inverse_transform(test_prediction)
r2_test = r2_score(y_test_actual, test_prediction_actual)
mse_test = mean_squared_error(y_test_actual, test_prediction_actual)
rmse_test = np.sqrt(mse_test)
mae_test = mean_squared_error(y_test_actual, test_prediction_actual)
percentage_deviations = ((y_test_actual-test_prediction_actual)/y_test_actual)*100
mpe_test = np.mean(percentage_deviations)
mape_test = np.mean(np.abs(percentage_deviations))

print("Final r2 on test set:", r2_test,"Final RMSE on test set:", rmse_test,
      "Final MAE on test set:", mae_test, "Final MPE on test set:", mpe_test,
      "Final MAPE on test set:", mape_test)
print("Final r2 on train set:", r2_train,"Final RMSE on train set:", rmse_train,
      "Final MAE on train set:", mae_train, "Final MPE on train set:", mpe_train,
      "Final MAPE on train set:", mape_train)


# Step 5: Visualize the result
test_prediction_actual=test_prediction_actual.reshape(-1,1)
## For test set
plt.figure(figsize=(8, 6))
plt.scatter(y_test_actual, test_prediction_actual, color='blue', alpha=0.5)
plt.plot([y_test_actual.min(), y_test_actual.max()], [y_test_actual.min(), y_test_actual.max()], color='red', linestyle='--')  # Diagonal line y=x
plt.title('Scatter Plot of Predicted vs. Actual Values')
plt.xlabel('Actual RTI') #NB: Use STI for STI models
plt.ylabel('Predicted RTI') #NB: Use STI for STI models
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)
# Annotate RMSE and R2
plt.annotate(f'RMSE: {rmse_test:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12)
plt.annotate(f'R2: {r2_test:.2f}', xy=(0.05, 0.90), xycoords='axes fraction', fontsize=12)
plt.show()

## For train set
train_prediction_actual=train_prediction_actual.reshape(-1,1)
plt.figure(figsize=(8, 6))
plt.scatter(y_train_actual, train_prediction_actual, color='blue', alpha=0.5)
plt.plot([y_train_actual.min(), y_train_actual.max()], [y_train_actual.min(), y_train_actual.max()], color='red', linestyle='--')  # Diagonal line y=x
plt.title('Scatter Plot of Predicted vs. Actual Values')
plt.xlabel('Actual RTI') #NB: Use STI for STI models
plt.ylabel('Predicted RTI') #NB: Use STI for STI models
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)
# Annotate RMSE and R2
plt.annotate(f'RMSE: {rmse_train:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12)
plt.annotate(f'R2: {r2_train:.2f}', xy=(0.05, 0.90), xycoords='axes fraction', fontsize=12)
plt.show()

## SHAP plot for the best model
### Initialize a Shapley explainer object with the model and data
explainer = shap.Explainer(model_name, data_set, check_additivity=False)
#NB: replace model_name with actual model name and data_set with either X_train or X_test
# Compute Shapley values for all samples
shap_values_train = explainer(X_train,check_additivity=False)
feature_labels = ['Permeability', 'Porosity', 'Salinity', 'Sgr', 'Depth','Thickness',
                  'Injection rate', 'Time']
# Summary plot
shap.summary_plot(shap_values_train, X_train, plot_type='bar', feature_names=feature_labels,
                  show=True)

# The end